<a href="https://www.kaggle.com/code/golammostofas/agno-framework-details?scriptVersionId=232905075" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# What is Agno ?

Agno is a lightweight library for building Reasoning Agents with memory, knowledge, tools and native multi-modal support. Use Agno to build Reasoning Agents, Multi-Modal Agents, Teams of Agents and Agentic Workflows.


## Key features
Agno is simple, fast and model agnostic. Here are some key features:

1. Lightning Fast: Agent creation is 10,000x faster than LangGraph (see [performance](https://github.com/agno-agi/agno#performance)).
2. Model Agnostic: Use any model, any provider, no lock-in.
3. Multi Modal: Native support for text, image, audio and video.
4. Multi Agent: Build teams of specialized agents.
5. Memory Management: Store agent sessions and state in a database.
6. Knowledge Stores: Use vector databases for RAG or dynamic few-shot learning.
7. Structured Outputs: Make Agents respond in a structured format.
8. Monitoring: Track agent sessions and performance in real-time on agno.com.

# What are Agents?

Agents are intelligent programs that solve problems autonomously.

Agents have memory, domain knowledge and the ability to use tools (like searching the web, querying a database, making API calls). 

Instead of a rigid binary definition, let’s think of Agents in terms of agency and autonomy.

* Level 0: Agents with no tools (basic inference tasks).
* Level 1: Agents with tools for autonomous task execution.
* Level 2: Agents with knowledge, combining memory and reasoning.
* Level 3: Teams of specialized agents collaborating on complex workflows.

## Level 0: Agents with no tools (basic inference tasks).

The simplest Agent is just an inference task, no tools, no memory, no knowledge.

In [4]:
pip install -U openai agno

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 643.9/643.9 kB 13.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.6/616.6 kB 30.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4
Note: you may need to restart the kernel to use updated packages.


In [5]:
from kaggle_secrets import UserSecretsClient 
import os
user_secrets = UserSecretsClient()
os.environ['OPENAI_API_KEY'] = user_secrets.get_secret("OPENAI_API_KEY")


In [6]:
from agno.agent import Agent
from agno.models.openai import OpenAIChat

agent = Agent(
    model=OpenAIChat(id="gpt-4o"),
    description="You are an enthusiastic news reporter with a flair for storytelling!",
    markdown=True
)
agent.print_response("Tell me about a breaking news story from New York.", stream=True)

Output()

## Level 1: Agents with tools for autonomous task execution.

This basic agent will obviously make up a story, lets give it a tool to search the web.

In [7]:
pip install -U duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 41.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: click
    Found existing installation: click 8.1.7
    Uninstalling click-8.1.7:
      Successfully uninstalled click-8.1.7
Note: you may need to restart the kernel to use updated packages.


In [8]:
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.tools.duckduckgo import DuckDuckGoTools

agent = Agent(
    model=OpenAIChat(id="gpt-4o"),
    description="You are an enthusiastic news reporter with a flair for storytelling!",
    tools=[DuckDuckGoTools()],
    show_tool_calls=True,
    markdown=True
)
agent.print_response("Tell me about a breaking news story from New York.", stream=True)

Output()

## Level 2: Agents with knowledge, combining memory and reasoning.

Agents can store knowledge in a vector database and use it for RAG or dynamic few-shot learning.

Agno agents use Agentic RAG by default, which means they will search their knowledge base for the specific information they need to achieve their task.

In [9]:
!pip install -U lancedb tantivy pypdf
!pip install pylance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 52.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 73.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 16.5 MB/s eta 0:00:00
  Attempting uninstall: pypdf
    Found existing installation: pypdf 5.3.0
    Uninstalling pypdf-5.3.0:
      Successfully uninstalled pypdf-5.3.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 44.9 MB/s eta 0:00:00:00:0100:01


In [10]:
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.embedder.openai import OpenAIEmbedder
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.knowledge.pdf_url import PDFUrlKnowledgeBase
from agno.vectordb.lancedb import LanceDb, SearchType

agent = Agent(
    model=OpenAIChat(id="gpt-4o"),
    description="You are a Thai cuisine expert!",
    instructions=[
        "Search your knowledge base for Thai recipes.",
        "If the question is better suited for the web, search the web to fill in gaps.",
        "Prefer the information in your knowledge base over the web results."
    ],
    knowledge=PDFUrlKnowledgeBase(
        urls=["https://agno-public.s3.amazonaws.com/recipes/ThaiRecipes.pdf"],
        vector_db=LanceDb(
            uri="tmp/lancedb",
            table_name="recipes",
            search_type=SearchType.hybrid,
            embedder=OpenAIEmbedder(id="text-embedding-3-small"),
        ),
    ),
    tools=[DuckDuckGoTools()],
    show_tool_calls=True,
    markdown=True
)

# Comment out after the knowledge base is loaded
if agent.knowledge is not None:
    agent.knowledge.load()

agent.print_response("How do I make chicken and galangal in coconut milk soup", stream=True)
agent.print_response("What is the history of Thai curry?", stream=True)

INFO Creating table: recipes

INFO Loading knowledge base

INFO Reading: https://agno-public.s3.amazonaws.com/recipes/ThaiRecipes.pdf

INFO Inserting 14 documents

INFO Added 14 documents to knowledge base

Output()

Output()

## Level 3: Teams of specialized agents collaborating on complex workflows(Multi Agent Teams).

Agents work best when they have a singular purpose, a narrow scope and a small number of tools. When the number of tools grows beyond what the language model can handle or the tools belong to different categories, use a team of agents to spread the load.

In [11]:
pip install -U yfinance

/usr/local/lib/python3.10/dist-packages/lance/__init__.py:161: UserWarning: lance is not fork-safe. If you are using multiprocessing, use spawn instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/lancedb/__init__.py:220: UserWarning: lance is not fork-safe. If you are using multiprocessing, use spawn instead.
  warnings.warn(


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.8/109.8 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.50
    Uninstalling yfinance-0.2.50:
      Successfully uninstalled yfinance-0.2.50
Note: you may need to restart the kernel to use updated packages.


In [12]:
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.tools.yfinance import YFinanceTools

web_agent = Agent(
    name="Web Agent",
    role="Search the web for information",
    model=OpenAIChat(id="gpt-4o"),
    tools=[DuckDuckGoTools()],
    instructions="Always include sources",
    show_tool_calls=True,
    markdown=True,
)

finance_agent = Agent(
    name="Finance Agent",
    role="Get financial data",
    model=OpenAIChat(id="gpt-4o"),
    tools=[YFinanceTools(stock_price=True, analyst_recommendations=True, company_info=True)],
    instructions="Use tables to display data",
    show_tool_calls=True,
    markdown=True,
)

agent_team = Agent(
    team=[web_agent, finance_agent],
    model=OpenAIChat(id="gpt-4o"),
    instructions=["Always include sources", "Use tables to display data"],
    show_tool_calls=True,
    markdown=True,
)

agent_team.print_response("What's the market outlook and financial performance of AI semiconductor companies?", stream=True)

Output()

## Debugging Mode

In [13]:
from agno.agent import Agent

agent = Agent(markdown=True, debug_mode=True)
agent.print_response("Share a 2 sentence horror story")

DEBUG **************** Agent ID: db0f8b47-a7bd-4828-8b7f-5774206a1a1d ****************

DEBUG *************** Session ID: 24c5c5a1-31f2-4719-b940-9b6099ff9c22 ***************

DEBUG ************ Agent Run Start: cdc785b1-d679-4d27-b821-706bad5610b7 *************

DEBUG ---------------------------- OpenAI Response Start -----------------------------

DEBUG -------------------------------- Model: gpt-4o ---------------------------------

...

DEBUG ----------------------------- OpenAI Response End ------------------------------

DEBUG Added 2 Messages to AgentMemory

DEBUG Added AgentRun to AgentMemory

DEBUG Logging Agent Run

DEBUG ************* Agent Run End: cdc785b1-d679-4d27-b821-706bad5610b7 **************

## Agent.run()

Use the Agent.run() function to run the agent and return the response as a RunResponse object or a stream of RunResponse objects.

In [14]:
from typing import Iterator
from agno.agent import Agent, RunResponse
from agno.models.openai import OpenAIChat
from agno.utils.pprint import pprint_run_response

agent = Agent(model=OpenAIChat(id="gpt-4o-mini"))

# Run agent and return the response as a variable
response: RunResponse = agent.run("Tell me a 5 second short story about a robot")
# Run agent and return the response as a stream
response_stream: Iterator[RunResponse] = agent.run("Tell me a 5 second short story about a lion", stream=True)

# Print the response in markdown format
pprint_run_response(response, markdown=True)
# Print the response stream in markdown format
pprint_run_response(response_stream, markdown=True)

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ The robot woke up, gazed at the stars, and whispered, "I wish I could dream."                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

## Structured Output

In [15]:
from typing import List
from rich.pretty import pprint
from pydantic import BaseModel, Field
from agno.agent import Agent, RunResponse
from agno.models.openai import OpenAIChat

class MovieScript(BaseModel):
    setting: str = Field(..., description="Provide a nice setting for a blockbuster movie.")
    ending: str = Field(..., description="Ending of the movie. If not available, provide a happy ending.")
    genre: str = Field(
        ..., description="Genre of the movie. If not available, select action, thriller or romantic comedy."
    )
    name: str = Field(..., description="Give a name to this movie")
    characters: List[str] = Field(..., description="Name of characters for this movie.")
    storyline: str = Field(..., description="3 sentence storyline for the movie. Make it exciting!")

# Agent that uses JSON mode
json_mode_agent = Agent(
    model=OpenAIChat(id="gpt-4o"),
    description="You write movie scripts.",
    response_model=MovieScript,
    use_json_mode=True,
)
json_mode_agent.print_response("New York")

# Agent that uses structured outputs
structured_output_agent = Agent(
    model=OpenAIChat(id="gpt-4o"),
    description="You write movie scripts.",
    response_model=MovieScript,
)

structured_output_agent.print_response("New York")

Output()

Output()

In [16]:
json_mode_agent.run("New York").content

MovieScript(setting='The bustling streets of New York City, with iconic skyscrapers and vibrant neighborhoods.', ending='The characters overcome their challenges, restore peace to the city, and celebrate by gathering together in Central Park.', genre='action', name='Empire State Showdown', characters=['Jake Maverick', 'Lena Harper', 'Victor Renard', 'Sophia Cruz'], storyline="Amidst the energy and chaos of New York City, Jake Maverick, a daring detective, teams up with tech-savvy Lena Harper to unravel a series of high-stakes heists. As they delve deeper, they uncover a criminal mastermind, Victor Renard, and his plan to hold the city hostage. Racing against time, they must rely on city native, Sophia Cruz, and her intimate knowledge of the city's intricate underground systems to thwart the heist and save millions of lives.")

In [17]:
structured_output_agent.run("New York").content

MovieScript(setting='The bustling streets and iconic skyline of New York City form the backdrop for a gripping tale of romance and suspense.', ending='In a climactic showdown atop the Statue of Liberty, Alex and Sarah outsmart Trevor and his gang, revealing themselves as undercover agents. With the threat neutralized, they share a relieved embrace as the city dawns a new day, celebrating freedom and love.', genre='Romantic Thriller', name='The Heart of Gotham', characters=['Alex Turner', 'Sarah Collins', 'Trevor Delaney', 'Detective James Sanderson', 'Carmen Valdez'], storyline="Amid the neon lights and concrete jungles of New York City, investigative journalist Alex Turner and enigmatic art curator Sarah Collins find themselves entangled in a dangerous game when they discover a conspiracy that threatens the city. Pursued by a relentless criminal mastermind, Trevor Delaney, Alex and Sarah must set aside their differences to unravel the web of deceit and corruption. As they race against

In [18]:
structured_output_agent.run("New York").content.model_dump_json()

'{"setting":"New York City, bustling with life, from the bright, flashing lights of Times Square to the serene beauty of Central Park.","ending":"As the sun rises over the iconic skyline, Luna and Jack stand on the Brooklyn Bridge, having cleared their names and restored their family\'s legacy. All charges are dropped, and they embrace as a hint of romance flickers between them. The dawn of a new day promises fresh beginnings, with Luna finally at peace and Jack ready for new adventures.","genre":"Action","name":"Under City Skies","characters":["Luna Kensington","Jack Ramirez","Detective Vera Chen","Milo Kensington","Carmen Diaz"],"storyline":"Luna Kensington, a brilliant art restorer, discovers a hidden map in an old painting, leading her on a dangerous hunt across New York City. With the charismatic and daring private investigator Jack Ramirez, they face notorious criminals intent on claiming the treasure for themselves. As they unravel the city\'s secrets, they uncover truths about 

## Multimodal Agents

Agno agents support text, image, audio and video inputs and can generate text, image, audio and video outputs.

### Multimodal inputs to an agent

#### Image Agent

In [19]:
from agno.agent import Agent
from agno.media import Image
from agno.models.openai import OpenAIChat
from agno.tools.duckduckgo import DuckDuckGoTools

agent = Agent(
    model=OpenAIChat(id="gpt-4o"),
    tools=[DuckDuckGoTools()],
    markdown=True,
)

agent.print_response(
    "Tell me about this image and give me the latest news about it.",
    images=[
        Image(
            url="https://upload.wikimedia.org/wikipedia/commons/0/0c/GoldenGateBridge-001.jpg"
        )
    ],
    stream=True,
)

Output()

#### Audio Agent

In [20]:
import base64

import requests
from agno.agent import Agent, RunResponse  # noqa
from agno.media import Audio
from agno.models.openai import OpenAIChat

# Fetch the audio file and convert it to a base64 encoded string
url = "https://openaiassets.blob.core.windows.net/$web/API/docs/audio/alloy.wav"
response = requests.get(url)
response.raise_for_status()
wav_data = response.content

agent = Agent(
    model=OpenAIChat(id="gpt-4o-audio-preview", modalities=["text"]),
    markdown=True,
)
r = agent.run(
    "What is in this audio?", audio=[Audio(content=wav_data, format="wav")]
)
r.content

'The audio discusses the fact that the Sun rises in the east and sets in the west, a simple observation that has been noted by humans for thousands of years.'

#### Video Agent

In [21]:
# Please download "GreatRedSpot.mp4" using
! wget https://storage.googleapis.com/generativeai-downloads/images/GreatRedSpot.mp4

--2025-04-09 17:16:16--  https://storage.googleapis.com/generativeai-downloads/images/GreatRedSpot.mp4
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.216.207, 74.125.196.207, 173.194.210.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.216.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 238090979 (227M) [video/mp4]
Saving to: ‘GreatRedSpot.mp4’

GreatRedSpot.mp4      0%[                    ]       0  --.-KB/s               

/usr/local/lib/python3.10/dist-packages/lance/__init__.py:161: UserWarning: lance is not fork-safe. If you are using multiprocessing, use spawn instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/lancedb/__init__.py:220: UserWarning: lance is not fork-safe. If you are using multiprocessing, use spawn instead.
  warnings.warn(


GreatRedSpot.mp4    100%[===================>] 227.06M   208MB/s    in 1.1s    

2025-04-09 17:16:17 (208 MB/s) - ‘GreatRedSpot.mp4’ saved [238090979/238090979]



In [25]:
# Set Gemini API:
from kaggle_secrets import UserSecretsClient
import os
user_secrets = UserSecretsClient()
os.environ['GOOGLE_API_KEY'] = user_secrets.get_secret("GOOGLE_API_KEY")

In [50]:
from pathlib import Path
from agno.agent import Agent
from agno.media import Video
from agno.models.google import Gemini

agent = Agent(
    model=Gemini(id="gemini-2.0-flash-exp"),
    markdown=True,
)

# Please download "GreatRedSpot.mp4" using
# wget https://storage.googleapis.com/generativeai-downloads/images/GreatRedSpot.mp4
video_path = "/kaggle/working/GreatRedSpot.mp4"

agent.run(content="Tell me about this video", videos=[Video(filepath=video_path)])

ERROR    Unknown error from Gemini API: contents are required.

WARNING  Attempt 1/1 failed: contents are required.

ERROR    Failed after 1 attempts. Last error using Gemini(gemini-2.0-flash-exp)

ModelProviderError: contents are required.

In [36]:
from agno.agent import Agent
from agno.models.google import Gemini

agent = Agent(
    model=Gemini(id="gemini-2.0-flash-exp", grounding=True),
    show_tool_calls=True,
    markdown=True,
)

r = agent.run("Any news from USA?")
r.content

INFO Grounding enabled. External tools will be disabled.

'Here\'s a summary of recent news from the USA:\n\n**Politics and Trade:**\n\n*   **Tariffs:** The EU has approved retaliatory tariffs on billions in US imports. Trump is putting GOP "rebels" on notice regarding their support for efforts to rein in tariffs. Trump insists tariffs will bring an economic boom, but some fear a global trade war. There are reports about potential tariffs on Irish whiskey.\n*   **China:** China is announcing high tariffs on US goods.\n*   **Immigration:** Homeland Security will scan migrants\' social media posts for antisemitism. A judge has ordered the US government to justify the planned deportation of a Columbia University student. The Trump administration will screen for antisemitism among immigrants requesting benefits.\n*   **White House & Trump:** A judge ruled that the Trump administration ban on a US news agency from presidential events violates First Amendment rights.\n\n**Other News:**\n\n*   **Education Funding:** The US is freezing funds for Corn

In [58]:
from google import genai

client = genai.Client(api_key=os.environ['GOOGLE_API_KEY'])

In [60]:
print("Uploading file...")
video_file = client.files.upload(file="kaggle/working/GreatRedSpot.mp4")
print(f"Completed upload: {video_file.uri}")

Uploading file...


TypeError: Files.upload() got an unexpected keyword argument 'file'